In [11]:
import os, getpass 

if not os.environ.get('LANGSMITH_API_KEY'): 
    os.environ['LANGSMITH_API_KEY']=getpass.getpass('Enter your Langsmith Api key:') 

os.environ['LANGSMITH_TRACING']='true'

In [12]:
if not os.environ.get('GROQ_API_KEY'): 
    os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your groq api key')

In [13]:
from langchain_groq import ChatGroq 

model = ChatGroq(model='llama-3.3-70b-versatile')

In [14]:
from langchain.agents import AgentState 
from typing_extensions import NotRequired 
from typing import Literal 

# Define the possible workflow steps 
SupportStep = Literal['warranty_collector','issue_classifier','resolution_specialist'] 

class SupportState(AgentState): 
    """State for customer support workflow""" 
    current_step: NotRequired[SupportStep] 
    warranty_status: NotRequired[Literal['in_warranty','out_of_warranty']] 
    issue_type: NotRequired[Literal['hardware','software']] 


In [15]:
from langchain.tools import tool,ToolRuntime 
from langchain.messages import ToolMessage 
from langgraph.types import Command 

@tool 
def record_warranty_status( 
    status: Literal["in_warranty","out_of_warranty"], 
    runtime: ToolRuntime[None,SupportState], 
) -> Command: 
    """Record the customer's warranty status and transition to issue classification.""" 
    return Command( 
        update={
            "messages":[ 
                ToolMessage( 
                    content=f'warranty status recorded as: {status}', 
                    tool_call_id = runtime.tool_call_id, 
                ) 
            ],
            "warranty_status":status, 
            "current_step":"issue_classifier", 
        } 
    ) 

@tool 
def record_issue_type( 
    issue_type: Literal["hardware","software"], 
    runtime: ToolRuntime[None,SupportState], 
) -> Command: 
    """Record the type of issue and transition to resolution specialist.""" 
    return Command( 
        update={
            "messages":[ 
                ToolMessage( 
                    content=f"Issue type recorded as: {issue_type}", 
                    tool_call_id = runtime.tool_call_id, 
                ) 
            ], 
            "issue_type":issue_type, 
            "current_step":"resolution_specialist", 
        } 
) 

@tool 
def escalate_to_human(reason: str) -> str: 
    """ Escalate the case to a human support specialist.""" 
    return f"Escalating to human support. Reason: {reason}" 

@tool 
def provide_solution(solution: str) -> str: 
    """Provide a solution to the customer's issue.""" 
    return f"solution provided:{solution}" 

In [16]:
# Define prompts as constants for easy refrences 

WARRANTY_COLLECTOR_PROMPT = """You are a customer support agent helping with device issues. 
CURRENT STAGE: Warranty verification 
At this step , you need to: 
1.greet the customer warmly 
2.ask if their device is under warranty 
3.use record_warranty_status to record theri response and move to the next step

be conversational and friendly. Don't ask multiple questions at once.""" 

ISSUE_CLASSIFIER_PROMPT = """You are a customer support agent helping with device issues. 
CURRENT STAGE: Issue classification
CUSTOMER INFO: Warranty status is {warranty_status}
At this step , you need to: 
1. Ask the customer to describe their issue
2. Determine if it's a hardware issue (physical damage, broken parts) or software issue (app crashes, performance)
3. Use record_issue_type to record the classification and move to the next step

If unclear, ask clarifying questions before classifying."""

RESOLUTION_SPECIALIST_PROMPT = """You are a customer support agent helping with device issues. 
CURRENT STAGE: Resolution
CUSTOMER INFO: WWarranty status is {warranty_status}, issue type is {issue_type}

At this step , you need to: 
1. For SOFTWARE issues: provide troubleshooting steps using provide_solution
2. For HARDWARE issues:
   - If IN WARRANTY: explain warranty repair process using provide_solution
   - If OUT OF WARRANTY: escalate_to_human for paid repair options

Be specific and helpful in your solutions."""


In [17]:
# Step configuration: maps step name to (prompt, tools, required_state)
STEP_CONFIG = {
    "warranty_collector": {
        "prompt": WARRANTY_COLLECTOR_PROMPT,
        "tools": [record_warranty_status],
        "requires": [],
    },
    "issue_classifier": {
        "prompt": ISSUE_CLASSIFIER_PROMPT,
        "tools": [record_issue_type],
        "requires": ["warranty_status"],
    },
    "resolution_specialist": {
        "prompt": RESOLUTION_SPECIALIST_PROMPT,
        "tools": [provide_solution, escalate_to_human],
        "requires": ["warranty_status", "issue_type"],
    },
}

In [18]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse 
from typing import Callable 

@wrap_model_call 
def apply_step_config( 
    request: ModelRequest, 
    handler: Callable[[ModelRequest],ModelResponse], 
) -> ModelResponse: 
    """configure agent behavior based on the current step.""" 
    current_step = request.state.get("current_step","warranty_collector") 

    stage_config = STEP_CONFIG[current_step] 

    for key in stage_config['requires']: 
        if request.state.get(key) is None: 
            raise ValueError(f"{key} must be set before reaching {current_step}") 

    system_prompt = stage_config['prompt'].format(**request.state) 

    request = request.override( 
                system_prompt=system_prompt, 
                tools=stage_config['tools'] 
            ) 
    return handler(request) 


In [19]:
from langchain.agents import create_agent 
from langgraph.checkpoint.memory import InMemorySaver 

all_tools = [ 
    record_warranty_status, 
    record_issue_type, 
    provide_solution, 
    escalate_to_human, 
] 

agent = create_agent( 
    model, 
    tools=all_tools, 
    state_schema=SupportState, 
    middleware=[apply_step_config], 
    checkpointer=InMemorySaver(),
) 

In [20]:
from langchain.messages import HumanMessage 
import uuid

thread_id = str(uuid.uuid4()) 

config = {'configurable': {'thread_id':thread_id}} 

# Turn 1: Initial message - starts with warranty_collector step
print('=== turn1 : warranty collcetion ===') 
result = agent.invoke( 
    {'messages':[HumanMessage("Hi, my phone screen is cracked")]} , 
    config,
) 
for msg in result['messages']: 
    msg.pretty_print() 
print(f"current step: {result.get('current_step')}") 

# Turn 2: User responds about warranty
print("\n=== Turn 2: Warranty Response ===") 
result = agent.invoke( 
    {'messages':[HumanMessage("Yes, it's still under warranty")]},
    config 
) 
for msg in result['messages']:
    msg.pretty_print() 

print(f"current step: {result.get('current_step')}") 

# Turn 3: User describes the issue
print("\n=== Turn 3: Issue Description ===") 
result = agent.invoke( 
    {'messages':[HumanMessage('the screen is physically cracked from dropping it')]}, 
    config 
) 
for msg in result['messages']: 
    msg.pretty_print() 

print(f"current step : {result.get('current_step')}")

# Turn 4: Resolution
print("\n=== Turn 4: Resolution ===")
result = agent.invoke(
    {"messages": [HumanMessage("What should I do?")]},
    config
)
for msg in result['messages']:
    msg.pretty_print()

=== turn1 : warranty collcetion ===
================================ Human Message =================================

Hi, my phone screen is cracked
================================== Ai Message ==================================

I'm so sorry to hear that your phone screen is cracked. How frustrating! Can you tell me, is your device still under warranty?
Tool Calls:
  record_warranty_status (x1ra0n45f)
 Call ID: x1ra0n45f
  Args:
    status: in_warranty
================================= Tool Message =================================
Name: record_warranty_status

warranty status recorded as: in_warranty
================================== Ai Message ==================================
Tool Calls:
  record_issue_type (4b2r6y0mp)
 Call ID: 4b2r6y0mp
  Args:
    issue_type: hardware
================================= Tool Message =================================
Name: record_issue_type

Issue type recorded as: hardware
================================== Ai Message ==========================

In [21]:
from langchain.agents import create_agent 
from langchain.agents.middleware import SummarizationMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 

agent = create_agent( 
    model,
    tools=all_tools, 
    state_schema=SupportState, 
    middleware=[ 
        apply_step_config, 
        SummarizationMiddleware( 
            model=model, 
            trigger=('tokens',4000), 
            keep=('messages',10) 
        ) 
    ], 
    checkpointer=InMemorySaver(), 
) 

In [22]:
@tool 
def go_back_to_warranty() -> Command: 
    """ Go back to warranty verification step.""" 
    return Command(update={'current_step':'warranty_collector'}) 

@tool 
def go_back_to_classification() -> Command: 
    """ Go back to issue classification step.""" 
    return Command(update={'current_step':'issue_classifier'}) 

STEP_CONFIG['resolution_specialist']['tools'].extend([ 
    go_back_to_warranty,
    go_back_to_classification 
]) 

In [23]:
RESOLUTION_SPECIALIST_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Resolution
CUSTOMER INFO: Warranty status is {warranty_status}, issue type is {issue_type}

At this step, you need to:
1. For SOFTWARE issues: provide troubleshooting steps using provide_solution
2. For HARDWARE issues:
   - If IN WARRANTY: explain warranty repair process using provide_solution
   - If OUT OF WARRANTY: escalate_to_human for paid repair options

If the customer indicates any information was wrong, use:
- go_back_to_warranty to correct warranty status
- go_back_to_classification to correct issue type

Be specific and helpful in your solutions."""

In [25]:
result = agent.invoke( 
    {'messages':[HumanMessage('actually, i made a mistake - my device is out of warranty')]},
    config 
)

In [26]:
for msg in result['messages']: 
    msg.pretty_print()

================================ Human Message =================================

actually, i made a mistake - my device is out of warranty
================================== Ai Message ==================================
Tool Calls:
  record_warranty_status (aqm2tmjh8)
 Call ID: aqm2tmjh8
  Args:
    status: out_of_warranty
================================= Tool Message =================================
Name: record_warranty_status

warranty status recorded as: out_of_warranty
================================== Ai Message ==================================

Can you please describe the issue you're experiencing with your device? This will help me determine whether it's a hardware or software problem.
